In [21]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

### VGG Trained with imagenet 

In [47]:
cnn = VGG16(weights = 'imagenet' , include_top = False ,input_shape = (150,150,3))

cnn.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [48]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [49]:
model = models.Sequential()
model.add(cnn) 
model.add(layers.Flatten()) 
model.add(layers.Dense(1024 , activation='relu'))
model.add(layers.Dense(512 ,activation ='relu'))
model.add(layers.Dense(2,activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              8389632   
_________________________________________________________________
dense_22 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 1026      
Total params: 23,630,146
Trainable params: 23,630,146
Non-trainable params: 0
_________________________________________________________________


In [50]:
print('Trainable weights before freezing the conv base:', len(model.trainable_weights))
cnn.trainable = False

print('Trainable weights After freezing the conv base:', len(model.trainable_weights))


Trainable weights before freezing the conv base: 32
Trainable weights After freezing the conv base: 6


In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_generator = ImageDataGenerator() 

train_data = train_generator.flow_from_directory('Train/', 
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



test_generator = ImageDataGenerator() 

test_data = test_generator.flow_from_directory('Test/', 
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [53]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size = train_data.n // train_data.batch_size

model.fit_generator(generator=train_data,
                    steps_per_epoch=step_size,
                    epochs=10)

Epoch 1/10
12/12 [==============================] - 26s 2s/step - loss: 24.8306 - accuracy: 0.7147
Epoch 2/10
12/12 [==============================] - 27s 2s/step - loss: 1.5155 - accuracy: 0.9457
Epoch 3/10
12/12 [==============================] - 26s 2s/step - loss: 0.8359 - accuracy: 0.9620
Epoch 4/10
12/12 [==============================] - 26s 2s/step - loss: 0.3700 - accuracy: 0.9837
Epoch 5/10
12/12 [==============================] - 26s 2s/step - loss: 0.5165 - accuracy: 0.9918
Epoch 6/10
12/12 [==============================] - 26s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 26s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 26s 2s/step - loss: 6.4788e-10 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 28s 2s/step - loss: 1.5873e-08 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 29s 2s/step - loss: 2.4295e-08 - accuracy: 1.000

In [55]:
loss , accuracy = model.evaluate(test_data, verbose=0)
print('Accuracy : ' + str(accuracy))

Accuracy : 0.9399999976158142


### Fine Tuning 

In [57]:
cnn.trainable = True

flag = False

for layer in cnn.layers:
    if layer.name == 'block5_conv1':
        flag = True
    
    if flag:
        layer.trainable = True
    else:
        layer.trainable = False

In [59]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size = train_data.n // train_data.batch_size

model.fit_generator(generator=train_data,
                    steps_per_epoch=step_size,
                    epochs=10)

/home/tirth/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
12/12 [==============================] - 31s 3s/step - loss: 51.2614 - accuracy: 0.8533
Epoch 2/10
12/12 [==============================] - 32s 3s/step - loss: 9.5569 - accuracy: 0.5788
Epoch 3/10
12/12 [==============================] - 31s 3s/step - loss: 101.3668 - accuracy: 0.5082
Epoch 4/10
12/12 [==============================] - 31s 3s/step - loss: 4.4808 - accuracy: 0.4918
Epoch 5/10
12/12 [==============================] - 31s 3s/step - loss: 0.6932 - accuracy: 0.5054
Epoch 6/10
12/12 [==============================] - 31s 3s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 7/10
12/12 [==============================] - 32s 3s/step - loss: 0.6932 - accuracy: 0.4918
Epoch 8/10
12/12 [==============================] - 31s 3s/step - loss: 0.6932 - accuracy: 0.4674
Epoch 9/10
12/12 [==============================] - 31s 3s/step - loss: 0.6932 - accuracy: 0.4728
Epoch 10/10
12/12 [==============================] - 34s 3s/step - loss: 0.6932 - accuracy: 0.4810


In [60]:
loss , accuracy = model.evaluate(test_data, verbose=0)
print('Accuracy : ' + str(accuracy))

Accuracy : 0.5
